In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
from pathlib import Path
import warnings
from IPython.display import clear_output

# Get the parent directory and add it to sys.path
parent_dir = Path().resolve().parent
sys.path.append(str(parent_dir))

warnings.filterwarnings("ignore")

from preprocess import pre_xenium

### 1. Preprocess internal data at each time step

In [10]:
import os

root_dir = "../../Jing/0029719"
folders = [folder for folder in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, folder))]
print(folders)
for folder in folders:
    path = os.path.join(root_dir, folder)
    print(path)
    pre_xenium(path)

['12h', '24h', '0h', '4h', '16h', '8h']
../../Jing/0029719/12h


100%|██████████| 469/469 [24:05<00:00,  3.08s/it]


../../Jing/0029719/24h


100%|██████████| 469/469 [20:51<00:00,  2.67s/it]


../../Jing/0029719/0h


...

100%|██████████| 469/469 [14:08<00:00,  1.81s/it]


../../Jing/0029719/16h


100%|██████████| 469/469 [21:25<00:00,  2.74s/it]


../../Jing/0029719/8h


100%|██████████| 469/469 [16:26<00:00,  2.10s/it]


### 2. Generate training sets

In [22]:
from data_builder_time_course import FeaturesWrapper, TimeSeriesBuilder
from data_builder import read_trans
import pandas as pd

# load transcripts and cell types
root_dir = "../../Jing/0029719/"
# folders = [folder for folder in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, folder))]
folders = ['0h', '4h', '8h', '12h', '16h', '24h']
save_path = root_dir + 'TimeSeries/'
gene_list = pd.read_csv(os.path.join(root_dir+folders[0], "features.tsv"), sep='\t', header=None)
gene_list = gene_list[gene_list[2] == "Gene Expression"][1] 
for i in range(len(gene_list)):
    gene = gene_list[i]
    clear_output(wait=True)
    print(f'Processing the {i+1}-th gene: {gene}')
    obs_list = []
    for folder in folders:
        folder_path = os.path.join(root_dir, folder)
        transcripts = read_trans(folder_path + '/MoleculesPerGene/' + gene + '.csv')
        time_stamp = int(folder.split("h")[0])
        obs = FeaturesWrapper(transcripts, time_stamp)
        obs_list.append(obs)
    # Build time series
    tsb = TimeSeriesBuilder(obs_list)
    tsb.run(save_path=save_path, gene=gene)

Processing the 469-th gene: ZC3HAV1
Building cell dictionaries
Build cell feature vectors
Building cell dictionaries
Build cell feature vectors
Building cell dictionaries
Build cell feature vectors
Building cell dictionaries
Build cell feature vectors
Building cell dictionaries
Build cell feature vectors
Building cell dictionaries
Build cell feature vectors


100%|██████████| 6368/6368 [00:00<00:00, 572167.00it/s]

6368 time-series samples of ZC3HAV1 generated
